In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [2]:
data = np.load('./data/data_pca_50_target.npz')
data.files

['arr_0', 'arr_1']

In [3]:
data.allow_pickle = True

In [4]:
X = data['arr_0'] # 50 componentes de dados do PCA
y = data['arr_1'] # variável alvo

In [5]:
X.shape,y.shape

((2381, 50), (2381,))

# Divisão de dados de treino e teste

In [6]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1904, 50) (477, 50) (1904,) (477,)


# Modelo Machine Learning

Grid Search para encontrar os melhores parâmetros

In [10]:
model_svc = SVC(probability=True)

param_grid = {'C':[0.5,1,10,20,30,50],
             'kernel':['rbf','poly'],
             'gamma':[0.1,0.05,0.01,0.001,0.002,0.005],
             'coef0':[0,1]}

In [11]:
model_grid = GridSearchCV(model_svc,
                          param_grid=param_grid,
                          scoring='accuracy',cv=3,verbose=2)

In [13]:
model_grid.fit(X_train,y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] END ..............C=0.5, coef0=0, gamma=0.1, kernel=rbf; total time=   0.7s
[CV] END ..............C=0.5, coef0=0, gamma=0.1, kernel=rbf; total time=   0.7s
[CV] END ..............C=0.5, coef0=0, gamma=0.1, kernel=rbf; total time=   0.7s
[CV] END .............C=0.5, coef0=0, gamma=0.1, kernel=poly; total time=   0.5s
[CV] END .............C=0.5, coef0=0, gamma=0.1, kernel=poly; total time=   0.5s
[CV] END .............C=0.5, coef0=0, gamma=0.1, kernel=poly; total time=   0.5s
[CV] END .............C=0.5, coef0=0, gamma=0.05, kernel=rbf; total time=   0.7s
[CV] END .............C=0.5, coef0=0, gamma=0.05, kernel=rbf; total time=   0.7s
[CV] END .............C=0.5, coef0=0, gamma=0.05, kernel=rbf; total time=   0.6s
[CV] END ............C=0.5, coef0=0, gamma=0.05, kernel=poly; total time=   0.5s
[CV] END ............C=0.5, coef0=0, gamma=0.05, kernel=poly; total time=   0.5s
[CV] END ............C=0.5, coef0=0, gamma=0.0

GridSearchCV(cv=3, estimator=SVC(probability=True),
             param_grid={'C': [0.5, 1, 10, 20, 30, 50], 'coef0': [0, 1],
                         'gamma': [0.1, 0.05, 0.01, 0.001, 0.002, 0.005],
                         'kernel': ['rbf', 'poly']},
             scoring='accuracy', verbose=2)

In [14]:
final_model = model_grid.best_estimator_

In [16]:
final_model.get_params()

{'C': 10,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 1,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 0.002,
 'kernel': 'poly',
 'max_iter': -1,
 'probability': True,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

# Avaliação do modelo


O modelo foi avaliado de 3 formas:<br>

- Classification Report
    - Precision, Recall, F1-Score
- Kappa Score
    - -ve (worst model)
    - 0 to 0.5 (bad model)
    - 0.5 to 0.7 (Good Model)
    - 0.7 to 0.9 (Excellent Model)
    - 0.9 to 1.0 (Perfect Model)
- AUC
    - Less than 0.5 (Worst Model)
    - 0.5 to 0.6 (Bad Model)
    - 0.6 to 0.8 (Good Model)
    - 0.8 to 0.9 (Excellent Model)
    - 0.9 to 1.0 (Perfect Model)

In [19]:
y_pred = final_model.predict(X_test) 

## Classification Report

In [20]:
cr = metrics.classification_report(y_test,y_pred,output_dict=True)
pd.DataFrame(cr).T

,precision,recall,f1-score,support
female,0.818182,0.894410,0.854599,322.000000
male,0.728000,0.587097,0.650000,155.000000
accuracy,0.794549,0.794549,0.794549,0.794549
macro avg,0.773091,0.740753,0.752300,477.000000
weighted avg,0.788877,0.794549,0.788115,477.000000


## Kappa Score

In [21]:
metrics.cohen_kappa_score(y_test,y_pred)

np.float64(0.5069507435924481)

É um bom modelo de acordo com o Kappa Score

## Area Under Cover (AUC)

In [22]:
metrics.roc_auc_score(np.where(y_test=="male",1,0),
                      np.where(y_pred=="male",1,0))

np.float64(0.7407533560408734)

É um bom modelo de acordo com o AUC

# Salvando o modelo

In [24]:
import pickle
pickle.dump(final_model,open('./model/model_svm.pickle',mode='wb'))